# Expedia Personalised Hotel Searches
VU Data Mining Techniques 2024 | Assignment 2 | Group 17

## Setup & Data Loading

In [1]:
import pandas as pd
from pprint import pprint

In [2]:
train_path = 'data/training_set_VU_DM.csv'
test_path = 'data/test_set_VU_DM.csv'

In [3]:
df = pd.read_csv(train_path)
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


## EDA
###### ToDo Jaime

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 54 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   srch_id                      int64  
 1   date_time                    object 
 2   site_id                      int64  
 3   visitor_location_country_id  int64  
 4   visitor_hist_starrating      float64
 5   visitor_hist_adr_usd         float64
 6   prop_country_id              int64  
 7   prop_id                      int64  
 8   prop_starrating              int64  
 9   prop_review_score            float64
 10  prop_brand_bool              int64  
 11  prop_location_score1         float64
 12  prop_location_score2         float64
 13  prop_log_historical_price    float64
 14  position                     int64  
 15  price_usd                    float64
 16  promotion_flag               int64  
 17  srch_destination_id          int64  
 18  srch_length_of_stay          int64  
 19  

In [5]:
df.describe()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
count,4.958347e+06,4.958347e+06,4.958347e+06,251866.000000,252988.000000,4.958347e+06,4.958347e+06,4.958347e+06,4.950983e+06,4.958347e+06,...,96174.000000,315348.000000,356422.000000,138515.000000,1.916654e+06,1.987503e+06,614730.000000,4.958347e+06,138390.000000,4.958347e+06
mean,1.663666e+05,9.953133e+00,1.753405e+02,3.374334,176.022659,1.739739e+02,7.007918e+04,3.180525e+00,3.777777e+00,6.346994e-01,...,17.250473,0.145969,0.083202,19.433267,-6.089936e-02,9.962752e-03,22.430384,4.474858e-02,386.283316,2.791051e-02
std,9.611223e+04,7.646890e+00,6.591625e+01,0.692519,107.254493,6.834525e+01,4.060992e+04,1.051024e+00,1.050329e+00,4.815144e-01,...,31.160313,0.578202,0.316722,54.370221,4.691723e-01,2.029142e-01,895.965854,2.067514e-01,821.190577,1.647165e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.410000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.000000,-1.000000,-1.000000,2.000000,-1.000000e+00,-1.000000e+00,2.000000,0.000000e+00,0.000000,0.000000e+00
25%,8.293600e+04,5.000000e+00,1.000000e+02,2.920000,109.810000,1.000000e+02,3.501000e+04,3.000000e+00,3.500000e+00,0.000000e+00,...,6.000000,0.000000,0.000000,7.000000,0.000000e+00,0.000000e+00,7.000000,0.000000e+00,124.000000,0.000000e+00
50%,1.665070e+05,5.000000e+00,2.190000e+02,3.450000,152.240000,2.190000e+02,6.963800e+04,3.000000e+00,4.000000e+00,1.000000e+00,...,11.000000,0.000000,0.000000,12.000000,0.000000e+00,0.000000e+00,11.000000,0.000000e+00,218.400000,0.000000e+00
75%,2.497240e+05,1.400000e+01,2.190000e+02,3.930000,213.490000,2.190000e+02,1.051680e+05,4.000000e+00,4.500000e+00,1.000000e+00,...,18.000000,1.000000,0.000000,20.000000,0.000000e+00,0.000000e+00,17.000000,0.000000e+00,429.790000,0.000000e+00
max,3.327850e+05,3.400000e+01,2.310000e+02,5.000000,1958.700000,2.300000e+02,1.408210e+05,5.000000e+00,5.000000e+00,1.000000e+00,...,1620.000000,1.000000,1.000000,9900.000000,1.000000e+00,1.000000e+00,149400.000000,1.000000e+00,159292.380000,1.000000e+00


In [6]:
df.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating        4706481
visitor_hist_adr_usd           4705359
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score      4640941
orig_destination_distance

If a visitor has a history of purchasing hotels (value for visitor_hist_starrating, visitor_hist_adr_usd) - it might be advantageous to higher weight their behaviour in our model. Because they are more likely to completely go through the booking process.

## Data Preprocessing
###### ToDo Ryan

### Dropping Features with Many Missing Values

In [7]:
# create a list of columns to drop where more than 50% of the data is missing
columns_to_drop = df.columns[df.isnull().mean() > 0.5].tolist()
pprint(f'Columns that are dropped due to excessive missing values: {columns_to_drop}')
df.drop(columns=columns_to_drop, inplace=True)

('Columns that are dropped due to excessive missing values: '
 "['visitor_hist_starrating', 'visitor_hist_adr_usd', "
 "'srch_query_affinity_score', 'comp1_rate', 'comp1_inv', "
 "'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', "
 "'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv', "
 "'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', "
 "'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', "
 "'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', "
 "'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', "
 "'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv', "
 "'comp8_rate_percent_diff', 'gross_bookings_usd']")


In [8]:
df.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
orig_destination_distance      1607782
random_bool                          0
click_bool                           0
booking_bool             

### Datatype Conversion

In [9]:
df['date_time'] = pd.to_datetime(df['date_time'])

boolean_features = ['prop_brand_bool', 'promotion_flag', 'srch_saturday_night_bool', 'random_bool']
for col in boolean_features:
    df[col] = df[col].astype('bool')
    
categorical_features = ['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_id', 'srch_destination_id']
for col in categorical_features:
    df[col] = df[col].astype('category')
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4958347 entries, 0 to 4958346
Data columns (total 26 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   srch_id                      category      
 1   date_time                    datetime64[ns]
 2   site_id                      category      
 3   visitor_location_country_id  category      
 4   prop_country_id              category      
 5   prop_id                      category      
 6   prop_starrating              int64         
 7   prop_review_score            float64       
 8   prop_brand_bool              bool          
 9   prop_location_score1         float64       
 10  prop_location_score2         float64       
 11  prop_log_historical_price    float64       
 12  position                     int64         
 13  price_usd                    float64       
 14  promotion_flag               bool          
 15  srch_destination_id          category      
 16  

### Handling Missing Values

In [10]:
# Check the dataset again
df.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7364
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1090348
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
orig_destination_distance      1607782
random_bool                          0
click_bool                           0
booking_bool             

In [11]:
# Fill missing values using a dictionary within the `fillna` method, avoiding the `inplace=True` argument to prevent annoying FutureWarnings
df = df.fillna({
    'prop_review_score': df['prop_review_score'].median(),  # Fill missing values for 'prop_review_score' using median as there's not too many missing values and it makes sense for a 5-star rating of properties
    'prop_location_score2': df['prop_location_score2'].mean(),  # For 'prop_location_score2', we fillna using the mean since it's a score
    'orig_destination_distance': df['orig_destination_distance'].mean()  # For 'orig_destination_distance' we will also take the mean distance as filler value
})

In [12]:
df.isnull().sum()

srch_id                        0
date_time                      0
site_id                        0
visitor_location_country_id    0
prop_country_id                0
prop_id                        0
prop_starrating                0
prop_review_score              0
prop_brand_bool                0
prop_location_score1           0
prop_location_score2           0
prop_log_historical_price      0
position                       0
price_usd                      0
promotion_flag                 0
srch_destination_id            0
srch_length_of_stay            0
srch_booking_window            0
srch_adults_count              0
srch_children_count            0
srch_room_count                0
srch_saturday_night_bool       0
orig_destination_distance      0
random_bool                    0
click_bool                     0
booking_bool                   0
dtype: int64

### Handling Outliers
Everything below here is still wip

ToDo! Based on what EDA shows we could use z-score or IQR

In [ ]:
# Example
Q1 = df['price_usd'].quantile(0.25)
Q3 = df['price_usd'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Apply the bounds to the data
df = df[(df['price_usd'] >= lower_bound) & (df['price_usd'] <= upper_bound)]

### Normalising/Standardising Numerical Features

Normalisation (scaling data between 0 and 1) or standardisation (shifting the distribution to have a mean of zero and a standard deviation of one) can be beneficial for algorithms that are sensitive to the scale of input data (like SVM or KNN).

In [ ]:
# Example
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Example: Standardizing 'price_usd'
scaler = StandardScaler()
df['price_usd'] = scaler.fit_transform(df[['price_usd']])

# Or for normalization
# scaler = MinMaxScaler()
# df['price_usd'] = scaler.fit_transform(df[['price_usd']])

### Encoding Categorical Variables

In [ ]:
# Example for 1-hot encoding a categorical feature
df = pd.get_dummies(df, columns=['prop_country_id'], drop_first=True)

### Feature Engineering

Ideas for this:
1. Time Features: Break down the date_time column into year, month, day, weekday, and hour components. Time could affect booking patterns.
2. Interaction Features: Create features that represent interactions between the customer’s historical preferences and property attributes, like the difference between the user’s average star rating and the property’s star rating.
3. Textual and Categorical Embeddings: If there are textual descriptions available or high-cardinality categorical variables, consider using embeddings or hashing techniques to reduce their dimensionality.

In [ ]:
# Create time-based features
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['weekday'] = df['date_time'].dt.weekday
df['hour'] = df['date_time'].dt.hour

# Interaction feature: difference between user's average star rating and property's star rating
df['starrating_diff'] = df['visitor_hist_starrating'] - df['prop_starrating']

# Re-check the dataset
df.head()